# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,salto del guaira,-24.0625,-54.3069,75.90,36,0,3.06,PY,1727654485
1,1,korkut,38.7391,41.7822,55.67,37,0,2.98,TR,1727654486
2,2,okhotsk,59.3833,143.3000,41.45,48,27,17.56,RU,1727654488
3,3,puerto natales,-51.7236,-72.4875,27.86,80,44,4.54,CL,1727654489
4,4,bikenibeu village,1.3673,173.1241,82.20,73,66,7.81,KI,1727654490


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',  # Size of points based on humidity
    color='City',  # Optional: color points based on humidity
    alpha=0.6,        # Transparency of points
    tiles='OSM',      # Use OpenStreetMap tiles for the background
    geo=True,          # Treat coordinates as geographic data
    title='Humidity of each city ',  # Title of the map
     frame_width=900,                        # Width of the plot
    frame_height=600  
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Filter for ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 40) &
    (city_data_df['Max Temp'] > 35)
    ]

# Drop any rows with null values
ideal_weather_df_cleaned = ideal_weather_df.dropna()
ideal_weather_df_cleaned

#Display a sample of the cleaned data
sample_data = ideal_weather_df_cleaned.sample(n=10) 

sample_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
491,491,sekovici,44.2986,18.8528,39.65,92,26,4.00,BA,1727655107
527,527,gadzhiyevo,69.2551,33.3362,35.89,87,100,18.12,RU,1727655150
98,98,egilsstadir,65.2653,-14.3948,37.29,97,100,8.75,IS,1727654607
29,29,iqaluit,63.7506,-68.5145,38.70,77,68,8.99,CA,1727654409
449,449,vorgashor,67.5833,63.9500,37.06,94,51,9.42,RU,1727655051
171,171,aasiaat,68.7098,-52.8699,37.92,95,100,28.57,GL,1727654704
560,560,harstad,68.7990,16.5293,36.79,96,84,18.99,NO,1727655191
162,162,ilulissat,69.2167,-51.1000,39.22,93,100,9.22,GL,1727654691
455,455,oberwart,47.2897,16.2060,38.84,72,62,4.90,AT,1727655060
315,315,polyarnyy,69.1989,33.4478,36.55,87,100,17.13,RU,1727654885


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng','Max Temp', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
sample_hotel_data = hotel_df.sample(n=10)
sample_hotel_data

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
54,keflavik,IS,64.0049,-22.5624,38.14,81,
416,saynshand,MN,44.8824,110.1163,37.99,74,
560,harstad,NO,68.7990,16.5293,36.79,96,
295,tayshet,RU,55.9342,98.0044,39.69,69,
565,hammerfest,NO,70.6634,23.6821,39.65,87,
106,vadso,NO,70.0744,29.7487,37.56,87,
143,vorkuta,RU,67.5000,64.0000,37.56,96,
369,isafjordur,IS,66.0755,-23.1240,35.64,75,
476,rovaniemi,FI,66.5000,25.7167,36.81,80,
162,ilulissat,GL,69.2167,-51.1000,39.22,93,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = { 'categories': 'accommodation.hotel', 
           'limit': 1,                           
           'apiKey': geoapify_key
        
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'# circle
    params["bias"] = f'proximity:{longitude},{latitude}'  

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json() 

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
iqaluit - nearest hotel: Frobisher Inn
keflavik - nearest hotel: Núpan Deluxe
egilsstadir - nearest hotel: Valaskjálf
vadso - nearest hotel: Scandic Vadsø
iskateley - nearest hotel: Авантаж
vorkuta - nearest hotel: "Мегаполис"
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
aasiaat - nearest hotel: SØMA
ust'-ilimsk - nearest hotel: Усть-Илимск
tromso - nearest hotel: Clarion Collection Hotel Aurora
iglino - nearest hotel: Ясная Поляна
tayshet - nearest hotel: Бирюса
polyarnyy - nearest hotel: No hotel found
aksuat - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
saynshand - nearest hotel: No hotel found
puerto aysen - nearest hotel: Hostal Dendal
springbok - nearest hotel: Elkoweru Guesthouse
vorgashor - nearest hotel: София
oberwart - nearest hotel: Gasthof zum Neubauer
rovaniemi - nearest hotel: Matkustajakoti Outa
sekovici - nearest hotel: No hotel found
gadzhiyevo - nearest hotel: Оазис
nyurba - nearest hotel: Марина
harstad 

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
29,iqaluit,CA,63.7506,-68.5145,38.70,77,Frobisher Inn
54,keflavik,IS,64.0049,-22.5624,38.14,81,Núpan Deluxe
98,egilsstadir,IS,65.2653,-14.3948,37.29,97,Valaskjálf
106,vadso,NO,70.0744,29.7487,37.56,87,Scandic Vadsø
116,iskateley,RU,67.6803,53.1512,39.16,88,Авантаж
143,vorkuta,RU,67.5000,64.0000,37.56,96,"""Мегаполис"""
162,ilulissat,GL,69.2167,-51.1000,39.22,93,Best Western Plus Hotel Ilulissat
171,aasiaat,GL,68.7098,-52.8699,37.92,95,SØMA
192,ust'-ilimsk,RU,58.0006,102.6619,35.80,97,Усть-Илимск
227,tromso,NO,69.6496,18.9570,35.62,87,Clarion Collection Hotel Aurora


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    size='Humidity',  # Size of points based on humidity
    color='City',  # Optional: color points based on humidity
    alpha=0.6,        # Transparency of points
    tiles='OSM',      # Use OpenStreetMap tiles for the background
    geo=True,          # Treat coordinates as geographic data
    title='Hotel Locations by City',  # Title of the map
    xlabel='Longitude',               
    ylabel='Latitude',                 
    frame_width=900,                       
    frame_height=600,
    hover_cols=['Country', 'Lat', 'Lng','Max Temp', 'Humidity', 'Hotel Name']
)

# Display the map
map_plot_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Hotel Name)